In [28]:
from __future__ import print_function
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import cifar10
import numpy as np
import os
import sys
from layer_utils import activation_quant, conv2d_noise, dense_noise
from resnet_model import resnet_v1, resnet_v2
from layers_numpy import *

In [2]:
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = False

n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

In [3]:
# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)


In [4]:
# x_train_3bit = quantize_rescale(quantize_unsigned(x_train, 3, 1.0), 3, 1.0)
x_test_3bit = quantize_rescale(quantize_unsigned(x_test, 3, 0.95), 3, 0.95)

In [5]:
activation_bits = 3
model_name = 'act%db_wnoise0.20' % activation_bits

# Model name, depth and version
model_type = 'ResNet%dv%d_%s' % (depth, version, model_name)
print(model_type)

ResNet20v1_act3b_wnoise0.20


In [29]:
K.clear_session()

if version == 2:
    model = resnet_v2(input_shape=input_shape,
                      depth=depth,
                      activation_bits=activation_bits,
                      weight_noise=0.00)
else:
    model = resnet_v1(input_shape=input_shape,
                      depth=depth,
                      activation_bits=activation_bits,
                      weight_noise=0.00)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])
model.summary()
print(model_type)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_noise (conv2d_noise)     (None, 32, 32, 12)   336         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 12)   48          conv2d_noise[0][0]               
__________________________________________________________________________________________________
activation_quant (activation_qu (None, 32, 32, 12)   1           batch_normalization[0][0]        
______________________________________________________________________________________________

In [30]:
# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'model_checkpoint\\checkpoint')
latest = tf.train.latest_checkpoint(save_dir)
print(latest)
model.load_weights(latest)

C:\Users\Weier Wan\Dropbox\cs231n\cifar10_resnet\model_checkpoint\checkpoint\Resnet20v1_act3b_wnoise0.20


In [31]:
# Score trained model.
scores = model.evaluate(x_test_3bit, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 27s 3ms/sample - loss: 0.4721 - acc: 0.83890s - loss: 0.4711 - acc: 0.83
Test loss: 0.47208574788570407
Test accuracy: 0.8389
